In [1]:
#pip install yfinance

In [2]:
# Import necessary libraries
import numpy as np
import pandas as pd
import yfinance as yf


In [3]:
# Fetch the EUR/CAD exchange rate data from Yahoo Finance
eur_cad = yf.download('EURCAD=X', start='2015-01-01', end='2022-03-09')

# Save the data to CSV file
eur_cad.to_csv('eur_cad.csv')

eur_cad.head()

eur_cad.info

[*********************100%***********************]  1 of 1 completed


<bound method DataFrame.info of                Open     High      Low    Close  Adj Close  Volume
Date                                                             
2015-01-01  1.40496  1.40496  1.40496  1.40496    1.40496       0
2015-01-02  1.40433  1.41050  1.40220  1.40445    1.40445       0
2015-01-05  1.40944  1.41125  1.40210  1.40850    1.40850       0
2015-01-06  1.40330  1.40613  1.39960  1.40330    1.40330       0
2015-01-07  1.40530  1.40894  1.39769  1.40550    1.40550       0
...             ...      ...      ...      ...        ...     ...
2022-03-02  1.41635  1.41667  1.40367  1.41643    1.41643       0
2022-03-03  1.40538  1.40538  1.39664  1.40531    1.40531       0
2022-03-04  1.40207  1.40314  1.39208  1.40218    1.40218       0
2022-03-07  1.38218  1.39280  1.37637  1.38239    1.38239       0
2022-03-08  1.39078  1.40387  1.38985  1.39050    1.39050       0

[1872 rows x 6 columns]>

# Column info
## Adj close

"Adj Close" column in the EUR/CAD CSV file provided by yfinance refers to the adjusted closing price of the EUR/CAD currency pair on a given day.

The adjusted closing price is calculated by taking into account any corporate actions or events that affect the price of the stock, such as stock splits, dividends, and stock buybacks. These events can cause significant fluctuations in the stock price, and the adjusted closing price provides a more accurate representation of the true value of the stock.

In the case of currency pairs, there are no corporate actions that affect the price, but the adjusted closing price may still be calculated to take into account any exchange rate adjustments made by the central bank or other regulatory authorities.

The "Adj Close" column can be useful for technical analysis and charting, as it provides a more accurate view of the historical price movements of the currency pair.

## Volume

the "Volume" column in the EUR/CAD CSV file provided by yfinance refers to the total number of units of the currency pair that were traded on a given day.

The volume can be used to gauge the level of market activity and liquidity in the currency pair. Higher volumes usually indicate a higher level of interest and participation from traders and investors, while lower volumes may indicate a lack of interest or a decrease in market activity.

In technical analysis, the volume is often used in conjunction with other indicators to identify potential price movements and trends in the market.


# Volumn column investigation 🔎

Thanks to the info, Volumn column seems to always be equal to 0, let's verify

In [4]:
mask = (eur_cad.Volume != 0)
volum_eur_cad = eur_cad.loc[mask]
volum_eur_cad.size

0

➡️ Volumn column can be removed

In [7]:
# drop the 'B' column
eur_cad = eur_cad.drop('Volume', axis=1) #axis=1 because we want to drop a column, axis=0 is for a row
print(eur_cad)

               Open     High      Low    Close  Adj Close
Date                                                     
2015-01-01  1.40496  1.40496  1.40496  1.40496    1.40496
2015-01-02  1.40433  1.41050  1.40220  1.40445    1.40445
2015-01-05  1.40944  1.41125  1.40210  1.40850    1.40850
2015-01-06  1.40330  1.40613  1.39960  1.40330    1.40330
2015-01-07  1.40530  1.40894  1.39769  1.40550    1.40550
...             ...      ...      ...      ...        ...
2022-03-02  1.41635  1.41667  1.40367  1.41643    1.41643
2022-03-03  1.40538  1.40538  1.39664  1.40531    1.40531
2022-03-04  1.40207  1.40314  1.39208  1.40218    1.40218
2022-03-07  1.38218  1.39280  1.37637  1.38239    1.38239
2022-03-08  1.39078  1.40387  1.38985  1.39050    1.39050

[1872 rows x 5 columns]


# General Data Preprocessing

In [8]:
# check for missing values
if eur_cad.isna().sum().sum() > 0:
    print("There are missing values in the dataset.")

➡️ No missing values

In [9]:
# check for duplicates
if eur_cad.duplicated().sum() > 0:
    print("There are duplicate rows in the dataset.")
    # if there are duplicates, you can remove those rows

➡️ No duplicates

In [13]:
# check for outliers
numeric_cols = eur_cad.select_dtypes(include=[np.number]).columns.tolist()
for col in numeric_cols:
    q1 = eur_cad[col].quantile(0.25)
    q3 = eur_cad[col].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    outliers = eur_cad[(eur_cad[col] < lower_bound) | (eur_cad[col] > upper_bound)]
    if outliers.shape[0] > 0:
        print(f"{outliers.shape[0]} outliers found in column {col}.")

21 outliers found in column Open.
14 outliers found in column High.
23 outliers found in column Low.
18 outliers found in column Close.
18 outliers found in column Adj Close.


In [14]:
outliers

,Open,High,Low,Close,Adj Close
Date,,,,,
2015-03-12,1.34360,1.35002,1.33947,1.34365,1.34365
2015-03-16,1.34350,1.35640,1.34210,1.34332,1.34332
2015-04-10,1.34180,1.34425,1.33400,1.34160,1.34160
2015-04-13,1.33382,1.33757,1.32980,1.33336,1.33336
2015-04-14,1.33120,1.33680,1.32550,1.33129,1.33129
2015-04-15,1.32869,1.33673,1.31140,1.32882,1.32882
2015-04-16,1.31350,1.32180,1.30760,1.31369,1.31369
2015-04-17,1.31240,1.31932,1.30260,1.31270,1.31270
2015-04-20,1.31931,1.31974,1.31030,1.31955,1.31955


➡️ Not seem to be anomalies